In [ ]:
import pystac_client
import odc.stac

In [ ]:
catalog = pystac_client.Client.open("https://explorer.dea.ga.gov.au/stac")

In [ ]:
odc.stac.configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
)

In [ ]:
# Set a bounding box
# [xmin, ymin, xmax, ymax] in latitude and longitude
bbox = [153.3, -27.5, 153.4, -27.6]

# Set a start and end date
start_date = "1988-01-01"
end_date = "1988-12-31"

# Set product ID as the STAC "collection"
collections = ["ga_ls_fc_pc_cyear_3"]

In [ ]:
# Build a query with the parameters above
query = catalog.search(
    bbox=bbox,
    collections=collections,
    datetime=f"{start_date}/{end_date}",
)

# Search the STAC catalog for all items matching the query
items = list(query.items())
print(f"Found: {len(items):d} datasets")

items

In [ ]:
ds = odc.stac.load(
             items=items,
             crs="EPSG:3577",
             lat=(bbox[1], bbox[3]),
             lon=(bbox[0], bbox[2]),
             time=("2015-01-01", "2018-12-31"))

ds


In [ ]:
# Print the loaded dataset
print("Dataset info:")
print(ds)

# Print the type of the bs_pc_10 data
print("\nType of bs_pc_10:")
print(type(ds.bs_pc_10))

# Print the data type (dtype) of the band values
print("\nData type of bs_pc_10 values:")
print(ds.bs_pc_10.dtype)

In [ ]:
ds.bs_pc_10[0]

In [ ]:
ds.bs_pc_10[1]

In [ ]:
ds.bs_pc_10[0][0]

In [ ]:
bbox = [146.29, -39.10, 146.39, -39.00]

# Set a start and end date
start_date = "2015-01-01"
end_date = "2020-12-31"

# Set product ID as the STAC "collection"
collections = ["ga_ls_fc_pc_cyear_3"]

# Build a query with the parameters above
query = catalog.search(
    bbox=bbox,
    collections=collections,
    datetime=f"{start_date}/{end_date}",
)

# Search the STAC catalog for all items matching the query
items = list(query.items())
print(f"Found: {len(items):d} datasets")


ds = odc.stac.load(
             items=items,
             crs="EPSG:3577",
             lat=(bbox[1], bbox[3]),
             lon=(bbox[0], bbox[2]),
             time=("2015-01-01", "2018-12-31"))

ds

In [ ]:
ds.bs_pc_10.__len__()

In [ ]:
ds.bs_pc_10[5]

In [ ]:
ds.bs_pc_10[5][5]

In [ ]:
ds.pv_pc_50[5][5]

In [ ]:
ds.pv_pc_50[5].mean().values

In [ ]:
for item in ds:

    if item == 'qa':
        continue

    mean = ds[item].mean().values

    print(f"Mean value of {item}: {mean}")